# Data Cleaning

In [131]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [132]:
df = pd.read_csv('BBC_country_data_regions.csv', sep = ';')

In [133]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
df.head()

,country,core_total,mitigation_total,adaptation_total,cross_cutting_total,other_total,core_multi,mitigation_multi,adaptation_multi,cross_cutting_multi,other_multi,ghg_lulucf_17,ghg_17,GDP_pc_18,population,region
0,Australia,"455,613,922.00",NaN,"15,611,850.00","247,509,858.00",NaN,"455,613,922.00",NaN,NaN,"145,694,970.00",NaN,"534,695.45","554,126.56",57354.96,24982688,Oceania
1,Austria,NaN,"171,622,069.89","32,572,011.17","78,174,346.48",NaN,NaN,"1,832,559.10",NaN,"64,137,273.66",NaN,"77,355.42","82,261.47",51453.15,8840521,Western Europe
2,Belgium,"359,973,745.47","6,969,203.64","71,549,171.36","16,714,481.05","25,760.96","359,973,745.47","1,345,613.50","18,140,259.74","180,975.31","25,760.96","114,280.49","114,539.90",47554.75,11427054,Western Europe
3,Canada,"140,038,232.35","170,608,276.52","63,238,766.01","211,061,790.43",NaN,"140,038,232.35",NaN,"1,549,811.50","45,499,758.85",NaN,"692,026.54","715,749.23",46454.74,37065178,North America
4,Croatia,"33,204.07",NaN,NaN,NaN,NaN,"33,204.07",NaN,NaN,NaN,NaN,"20,253.17","25,019.75",15014.08,4087843,Southern Europe


In [134]:
print(type(df.core_total[0]))


<class 'str'>


In [135]:
#list of columns to convert
columns = df.columns[1:-3]
columns

Index(['core_total', 'mitigation_total', 'adaptation_total',
       'cross_cutting_total', 'other_total', 'core_multi', 'mitigation_multi',
       'adaptation_multi', 'cross_cutting_multi', 'other_multi',
       'ghg_lulucf_17', 'ghg_17'],
      dtype='object')

In [136]:
#replace NANs, converting to number
for col in columns:
    df[col] = df[col].str.replace(',','')
    df[col] = df[col].fillna(0)
    df[col] = pd.to_numeric(df[col])
    


In [137]:
df.head()

,country,core_total,mitigation_total,adaptation_total,cross_cutting_total,other_total,core_multi,mitigation_multi,adaptation_multi,cross_cutting_multi,other_multi,ghg_lulucf_17,ghg_17,GDP_pc_18,population,region
0,Australia,455613922.00,0.00,15611850.00,247509858.00,0.00,455613922.00,0.00,0.00,145694970.00,0.00,534695.45,554126.56,57354.96,24982688,Oceania
1,Austria,0.00,171622069.89,32572011.17,78174346.48,0.00,0.00,1832559.10,0.00,64137273.66,0.00,77355.42,82261.47,51453.15,8840521,Western Europe
2,Belgium,359973745.47,6969203.64,71549171.36,16714481.05,25760.96,359973745.47,1345613.50,18140259.74,180975.31,25760.96,114280.49,114539.90,47554.75,11427054,Western Europe
3,Canada,140038232.35,170608276.52,63238766.01,211061790.43,0.00,140038232.35,0.00,1549811.50,45499758.85,0.00,692026.54,715749.23,46454.74,37065178,North America
4,Croatia,33204.07,0.00,0.00,0.00,0.00,33204.07,0.00,0.00,0.00,0.00,20253.17,25019.75,15014.08,4087843,Southern Europe


In [138]:
#import spending data
spending = pd.read_csv('spending.csv', sep = ';',usecols = ['year', 'USD', 'type'])


In [139]:
#convert to int
spending['USD'] = spending['USD'].str.replace(',','.')
spending['USD'] = pd.to_numeric(spending['USD'])

In [140]:
#import instrument data
instrument = pd.read_csv('instrument.csv', sep= ';', usecols = ['year', 'USD', 'instrument'])
#convert to int
instrument['USD'] = instrument['USD'].str.replace(',','.')
instrument['USD'] = pd.to_numeric(instrument['USD'])

In [141]:
instrument.head()

,year,USD,instrument
0,2013,10.30,grant
1,2013,19.80,loan
2,2013,7.80,other
3,2013,37.90,total
4,2014,9.80,grant


In [142]:
#import recipient data
recipients = pd.read_csv('recipients.csv', sep = ';')
#convert to int
recipients['USD'] =recipients['USD'].str.replace(',','.')
recipients['USD'] = pd.to_numeric(recipients['USD'])

In [143]:
recipients.head()

,group,USD
0,LIC,5.40
1,LMIC,28.10
2,UMIC,20.00
3,HIC,1.70
4,unallocable,14.30


In [144]:
#import comparison data
comparison = pd.read_csv('comparison.csv', sep = ';')
comparison['USD_revenue'] = pd.to_numeric(comparison['USD_revenue'])

In [145]:
comparison = comparison[:3]
comparison['actor'] = ['Climate finance 2019', 'Climate finance goal', 'Covid-19 response']
comparison.loc[0] = ['Climate finance 2019', 79.6]
comparison

,actor,USD_revenue
0,Climate finance 2019,79.60
1,Climate finance goal,100.00
2,Covid-19 response,11000.00


# Visualising

In [146]:
#trial with comparison data and bokeh
from bokeh.io import output_file, show, save
from bokeh.models import ColumnDataSource, Legend, Label
from bokeh.plotting import figure
from bokeh.palettes import viridis
#import bokeh libraries
from bokeh.io import output_file, show, save
from bokeh.models import ColumnDataSource, Legend, Label, BoxAnnotation, HoverTool, CategoricalColorMapper
from bokeh.models import LinearColorMapper,ColorBar, BasicTicker, PrintfTickFormatter, BasicTickFormatter, Title
from bokeh.models import NumeralTickFormatter, WheelZoomTool, BoxZoomTool, PrintfTickFormatter
from bokeh.plotting import figure
from bokeh.palettes import viridis, inferno, d3, Turbo256
from math import pi
from bokeh.transform import cumsum, transform
from bokeh.layouts import row,column
from bokeh.io import export_png

In [292]:
#visualise comparison between actors

# Income is based on Wikipedia, which sources their annual reports

#sort actors according to income
actor = list(comparison['actor'])
revenue = list(comparison['USD_revenue'])

sorted_actors = sorted(actor, key=lambda x: revenue[actor.index(x)])
sorted_revenue = sorted(revenue)

#colour palette
colors = viridis(6)

#set source and tooltips
source = ColumnDataSource(data=dict(actor=sorted_actors, revenue=sorted_revenue, color=colors[3:6]))

TOOLTIPS = [
    ("", "@actor"),
    ("USD", "@revenue{($ 0.0)} bn"),
]


#create figure
p = figure(x_range=sorted_actors, y_range=(-2000,15000), 
           title="How much is $100 billion?", tooltips = TOOLTIPS, y_axis_label='$ billion', 
           toolbar_location=None, width = 465, height = 465)

p.sizing_mode ='stretch_both'
#add empty legend
#p.add_layout(Legend(), 'right')

#set bars
p.vbar(x='actor', top='revenue', width=0.5, color='color',source=source)


p.xgrid.grid_line_color = None

p.xaxis.major_label_orientation = 120

#add data sources below charts
label_opts = dict(x = 0, y = 0, x_units = 'screen', y_units ='screen')
msg = 'Data Sources: OECD, IMF'
caption1 = Label(text=msg, **label_opts, text_font_size = '8pt', text_font_style = 'italic')
p.add_layout(caption1, 'below')
msg2 ='The Independent Expert Group on Climate Finance'
#caption3=Label(text = msg2, **label_opts, text_font_size = '8pt', text_font_style = 'italic')
#p.add_layout(caption3, 'below')
#
p.toolbar.active_drag = None

p.left[0].formatter.use_scientific = False
p.yaxis[0].formatter = NumeralTickFormatter(format="0,0")
output_file("comparison.html")
show(p)

save(p)

'/Users/kajsarosenblad/Documents/private/Skrivet/BBC/100B/Data/comparison.html'

In [148]:
#visualise the historical trend and public private division
spending

,year,USD,type
0,2013,22.50,bilateral_public
1,2013,12.80,private
2,2013,15.50,multi_public
3,2013,1.60,export_credits
4,2013,52.40,total
5,2014,23.10,bilateral_public
6,2014,16.70,private
7,2014,20.40,multi_public
8,2014,1.60,export_credits
9,2014,61.80,total


In [149]:
#trial = spending[spending['type'] != 'total']
trial = spending[spending['year']!= 2015]
trial.head()

,year,USD,type
0,2013,22.50,bilateral_public
1,2013,12.80,private
2,2013,15.50,multi_public
3,2013,1.60,export_credits
4,2013,52.40,total


In [150]:
#try to make pivot table
pt = pd.pivot_table(trial, values='USD', index=['year'],
                    columns=['type'])

first_col = pt.pop('private')



In [151]:
pt

type,bilateral_public,export_credits,multi_public,total
year,,,,
2013,22.50,1.60,15.50,52.40
2014,23.10,1.60,20.40,61.80
2016,28.00,1.50,18.90,58.50
2017,27.00,2.10,27.50,71.10
2018,32.00,2.10,29.60,78.30
2019,28.80,2.60,34.10,79.60


In [152]:
pt.insert(0,'private', first_col)


In [274]:
pt['public'] = pt['bilateral_public'] + pt['multi_public']
pt = pt.drop(['bilateral_public', 'multi_public', 'total'], axis = 1)


KeyError: 'bilateral_public'

In [279]:
pt

type,private,export_credits,public
year,,,
2013,12.80,1.60,38.00
2014,16.70,1.60,43.50
2016,10.10,1.50,46.90
2017,14.50,2.10,54.50
2018,14.60,2.10,61.60
2019,14.00,2.60,62.90


In [272]:
pt_sum = pt.cumsum(axis = 1)

In [288]:
pt_sum.columns = ['private_sum', 'ex_sum', 'public_sum']

In [289]:
total = pd.concat([pt, pt_sum], axis = 1)
total

,private,export_credits,public,private_sum,ex_sum,public_sum
year,,,,,,
2013,12.80,1.60,38.00,12.80,14.40,52.40
2014,16.70,1.60,43.50,16.70,18.30,61.80
2016,10.10,1.50,46.90,10.10,11.60,58.50
2017,14.50,2.10,54.50,14.50,16.60,71.10
2018,14.60,2.10,61.60,14.60,16.70,78.30
2019,14.00,2.60,62.90,14.00,16.60,79.50


In [293]:
from bokeh.models import BoxAnnotation, Label, HoverTool
output_file("spending.html")


#set up figure
p1 = figure(title="Historical spending",
           x_axis_label='Year', y_axis_label='$ billion',
           x_range = (2012.5,2019.5),y_range=(0,110), toolbar_location=None, sizing_mode='stretch_both')

p1.add_layout(Title(text="Data gap in 2015 due to updated measurement methods.", align="left", text_font_size = '8pt', text_font_style = 'italic'), "above")

#add empty legend
#p1.add_layout(Legend(), 'right')

#add three stacks + 1 tool
first = p1.vbar(x='year', bottom=0, top='private_sum', source = total, width=0.9, color=colors[0], legend_label='Private')

p1.add_tools(HoverTool(renderers=[first], tooltips=[('Private finance',"@private{$0.0} bn")],mode='mouse'))

second = p1.vbar(x='year', bottom='private_sum', top='ex_sum', source = total, width=0.9, color=colors[3], legend_label='Export Credits')
p1.add_tools(HoverTool(renderers=[second], tooltips=[('Export credits',"@export_credits{$0.0} bn")],mode='mouse'))

third =  p1.vbar(x='year', bottom='ex_sum', top='public_sum', source = total, width=0.9, color = colors[5], legend_label='Public')
p1.add_tools(HoverTool(renderers=[third], tooltips=[('Public finance',"@public{$0.0} bn")],mode='mouse'))


#add 100 billion line
low_box = BoxAnnotation(top=100.2,bottom = 99.8, fill_color='red', level ='underlay')
p1.add_layout(low_box)
p1.xaxis.major_label_orientation = 120

#add legend
#p1.legend.orientation = "horizontal"
p1.legend.location = "top_right"

#add manual labels
#p1.add_layout(Label(x=497, y=25, x_units='screen', text='$78.9 Billion', render_mode='css', text_font_size ='10pt'))
#p1.add_layout(Label(x=404, y=25, x_units='screen', text='$71.2 Billion', render_mode='css', text_font_size ='10pt'))
#p1.add_layout(Label(x=318, y=25, x_units='screen', text='$58.6 Billion', render_mode='css', text_font_size ='10pt'))
#p1.add_layout(Label(x=238, y=25, x_units='screen', text='Data Gap',
                  # render_mode='css', text_font_size ='10pt'))
#p1.add_layout(Label(x=142, y=25, x_units='screen', text='$61.8 Billion', render_mode='css', text_font_size ='10pt'))
#p1.add_layout(Label(x=52, y=25, x_units='screen', text='$52.4 Billion', render_mode='css', text_font_size ='10pt'))
p1.add_layout(Label(x_offset = 20, y=101, x_units='screen', text='$100 Billion', render_mode='css', text_font_size ='10pt'))

#add data sources below charts
label_opts = dict(x = 0, y = 5, x_units = 'screen', y_units ='screen')
msg = 'Data Source: OECD'
caption1 = Label(text=msg, **label_opts, text_font_size = '8pt', text_font_style = 'italic')
p1.add_layout(caption1, 'below')

msg2 = 'Data Gap in 2015 due to the implementation of enhanced measurement methods.'
caption2 = Label(text=msg2, **label_opts, text_font_size = '8pt', text_font_style = 'italic')
#p1.add_layout(caption2, 'below')

p1.toolbar.active_drag = None

output_file("spending.html")
show(p1)

save(p1)

'/Users/kajsarosenblad/Documents/private/Skrivet/BBC/100B/Data/spending.html'

In [157]:
#loans vs grants
instrument.head()

,year,USD,instrument
0,2013,10.30,grant
1,2013,19.80,loan
2,2013,7.80,other
3,2013,37.90,total
4,2014,9.80,grant


In [158]:
#try to make pivot table
table1 = pd.pivot_table(instrument, values='USD', index=['year'],
                    columns=['instrument']).reset_index()

In [159]:
table1

instrument,year,grant,loan,other,total
0,2013,10.30,19.80,7.80,37.90
1,2014,9.80,28.10,5.60,43.50
2,2015,10.20,30.70,1.20,42.10
3,2016,12.00,33.60,1.30,46.90
4,2017,12.80,39.80,1.90,54.50
5,2018,12.30,46.30,3.60,62.20


In [160]:
row = [2019, 16.7,44.5,1.7, 62.9]
table1.loc[6] = row
table1

instrument,year,grant,loan,other,total
0,2013.00,10.30,19.80,7.80,37.90
1,2014.00,9.80,28.10,5.60,43.50
2,2015.00,10.20,30.70,1.20,42.10
3,2016.00,12.00,33.60,1.30,46.90
4,2017.00,12.80,39.80,1.90,54.50
5,2018.00,12.30,46.30,3.60,62.20
6,2019.00,16.70,44.50,1.70,62.90


In [161]:
output_file('loangrants.html')

In [162]:
#datasource
source = ColumnDataSource(data = table1)

#set up plot
plot = figure(title="Type of public finance", x_axis_label='Year', y_axis_label='$ billion', 
            sizing_mode='stretch_both', toolbar_location=None)

#add empty legend
#plot.add_layout(Legend(), 'right')


plot2 =plot.line(x='year',y= 'loan',source=source, color = colors[2],line_width = 7, legend_label="Loan")
plot.add_tools(HoverTool(renderers=[plot2], tooltips=[('Loan',"@loan{$ 0.0} bn")],mode='mouse'))

plot1 =plot.line(x='year',y= 'grant',source=source, color = colors[0], line_width = 7, legend_label="Grant")
plot.add_tools(HoverTool(renderers=[plot1], tooltips=[('Grant',"@grant{$ 0.0} bn")],mode='mouse'))


plot3 =plot.line(x='year',y= 'other',source=source, color = colors[4], line_width = 7, legend_label="Other")
plot.add_tools(HoverTool(renderers=[plot3], tooltips=[('Other',"@other{$ 0.0} bn")],mode='mouse'))

#plot.legend.orientation = "horizontal"
plot.legend.location = "top_left"

#add data sources below charts
label_opts = dict(x = 0, y = 0, x_units = 'screen', y_units ='screen')
msg = 'Data Source: OECD'
caption1 = Label(text=msg, **label_opts, text_font_size = '8pt', text_font_style = 'italic')
plot.add_layout(caption1, 'below')
plot.xaxis.major_label_orientation = 120
plot.toolbar.active_drag = None

output_file('loangrants.html')
show(plot)

save(plot)

'/Users/kajsarosenblad/Documents/private/Skrivet/BBC/100B/Data/loangrants.html'

In [163]:
#circle graph
recipients['group'] = ['Low-Income Countries', 'Lower Middle Income Countries', 'Upper Middle Income Countries', 'High-Income Countries', 'Unallocable']
recipients.head()

,group,USD
0,Low-Income Countries,5.40
1,Lower Middle Income Countries,28.10
2,Upper Middle Income Countries,20.00
3,High-Income Countries,1.70
4,Unallocable,14.30


In [164]:
#Circle graph

output_file('piechart.html')

from math import pi
from bokeh.transform import cumsum

x = pd.Series(recipients.USD.values,index=recipients.group).to_dict()

data = pd.Series(x).reset_index(name='value').rename(columns={'index':'country'})
data['angle'] = data['value']/data['value'].sum() * 2*pi
data['color'] = colors[:5]

p3 = figure(sizing_mode='stretch_both',title="2016-18 average per income group", toolbar_location=None,
           tools="hover", tooltips="@country: @value{$0.0} bn ")
#add empty legend
#p3.add_layout(Legend(), 'below')
#p3.legend.orientation = 'horizontal'

p3.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='country', source=data)

p3.axis.axis_label=None
p3.axis.visible=False
p3.grid.grid_line_color = None

#add data sources below charts
label_opts = dict(x = 0, y = 0, x_units = 'screen', y_units ='screen')
msg = 'Data Source: OECD'
caption1 = Label(text=msg, **label_opts, text_font_size = '8pt', text_font_style = 'italic')
p3.add_layout(caption1, 'below')


show(p3)
save(p3)

'/Users/kajsarosenblad/Documents/private/Skrivet/BBC/100B/Data/piechart.html'

In [165]:
#bubble chart
df.head()

,country,core_total,mitigation_total,adaptation_total,cross_cutting_total,other_total,core_multi,mitigation_multi,adaptation_multi,cross_cutting_multi,other_multi,ghg_lulucf_17,ghg_17,GDP_pc_18,population,region
0,Australia,455613922.00,0.00,15611850.00,247509858.00,0.00,455613922.00,0.00,0.00,145694970.00,0.00,534695.45,554126.56,57354.96,24982688,Oceania
1,Austria,0.00,171622069.89,32572011.17,78174346.48,0.00,0.00,1832559.10,0.00,64137273.66,0.00,77355.42,82261.47,51453.15,8840521,Western Europe
2,Belgium,359973745.47,6969203.64,71549171.36,16714481.05,25760.96,359973745.47,1345613.50,18140259.74,180975.31,25760.96,114280.49,114539.90,47554.75,11427054,Western Europe
3,Canada,140038232.35,170608276.52,63238766.01,211061790.43,0.00,140038232.35,0.00,1549811.50,45499758.85,0.00,692026.54,715749.23,46454.74,37065178,North America
4,Croatia,33204.07,0.00,0.00,0.00,0.00,33204.07,0.00,0.00,0.00,0.00,20253.17,25019.75,15014.08,4087843,Southern Europe


In [166]:
#add column of total multilateral and bilateral support
total_support = df['core_total'] + df['mitigation_total']+df['adaptation_total']+df['cross_cutting_total'] + df['other_total']

df['total_support'] = total_support

#add column for standardised gdp per capita (to make colouring easier)
df['GDP_pc_18_stand'] = df['GDP_pc_18']/10000



In [167]:
#add column emission per capita
per_capita = (df['ghg_lulucf_17'])/df['population']*1000

df['per_capita'] = per_capita

In [168]:
#standard lulcf for size of scatter 
#df['stand_ghg'] = df['ghg_lulucf_17'] /18000

In [169]:
df

,country,core_total,mitigation_total,adaptation_total,cross_cutting_total,other_total,core_multi,mitigation_multi,adaptation_multi,cross_cutting_multi,other_multi,ghg_lulucf_17,ghg_17,GDP_pc_18,population,region,total_support,GDP_pc_18_stand,per_capita
0,Australia,455613922.00,0.00,15611850.00,247509858.00,0.00,455613922.00,0.00,0.00,145694970.00,0.00,534695.45,554126.56,57354.96,24982688,Oceania,718735630.00,5.74,21.40
1,Austria,0.00,171622069.89,32572011.17,78174346.48,0.00,0.00,1832559.10,0.00,64137273.66,0.00,77355.42,82261.47,51453.15,8840521,Western Europe,282368427.54,5.15,8.75
2,Belgium,359973745.47,6969203.64,71549171.36,16714481.05,25760.96,359973745.47,1345613.50,18140259.74,180975.31,25760.96,114280.49,114539.90,47554.75,11427054,Western Europe,455232362.48,4.76,10.00
3,Canada,140038232.35,170608276.52,63238766.01,211061790.43,0.00,140038232.35,0.00,1549811.50,45499758.85,0.00,692026.54,715749.23,46454.74,37065178,North America,584947065.31,4.65,18.67
4,Croatia,33204.07,0.00,0.00,0.00,0.00,33204.07,0.00,0.00,0.00,0.00,20253.17,25019.75,15014.08,4087843,Southern Europe,33204.07,1.50,4.95
5,Cyprus,468202.37,0.00,0.00,0.00,0.00,468202.37,0.00,0.00,0.00,0.00,8429.23,8963.24,29089.47,1189262,Southern Europe,468202.37,2.91,7.09
6,Czechia,7231371.00,1657304.00,4792746.00,2046140.00,0.00,7231371.00,0.00,0.00,1189931.00,0.00,126540.11,128675.05,23419.74,10629928,Eastern Europe,15727561.00,2.34,11.90
7,Denmark,258702416.71,92916901.74,66160009.12,74371086.71,0.00,258702416.71,16539806.13,29543611.92,15330642.38,0.00,50582.08,47611.52,61598.54,5793636,Northern Europe,492150414.28,6.16,8.73
8,Estonia,444181.22,236168.63,36929.06,862682.17,0.00,444181.22,104278.68,0.00,13672.74,0.00,19087.14,20879.88,23159.39,1321977,Northern Europe,1579961.08,2.32,14.44
9,Finland,633419141.96,14598371.65,5311155.08,35071380.37,0.00,633419141.96,4027664.35,2478460.99,18085069.93,0.00,34956.41,55334.13,50013.29,5515525,Northern Europe,688400049.06,5.00,6.34


In [170]:
gdp = pd.read_csv('gdp.csv',skiprows=[i for i in range(0,3)])
#print(gdp['Country Name'].unique())

spare = gdp[(gdp['Country Name'].isin(['Russian Federation', 'Slovak Republic', 'Czech Republic']))]

gdp = gdp[(gdp['Country Name'].isin(list(df.country)))]


gdp = pd.concat([gdp, spare])

gdp = gdp.sort_values('Country Name')

gdp_18 = list(gdp['2018'])

df['gdp_18'] = gdp_18

In [206]:
ghg = pd.read_csv('gdp.csv',skiprows=[i for i in range(0,3)])

spare = ghg[(ghg['Country Name'].isin(['Russian Federation', 'Slovak Republic', 'Czech Republic']))]

ghg = ghg[(ghg['Country Name'].isin(list(df.country)))]

ghg = pd.concat([ghg, spare])

ghg = ghg.sort_values('Country Name')

ghg_18 = list(ghg['2018'])

df['ghg_18'] = ghg_18

#standard lulcf for size of scatter 
df['stand_ghg'] = df['ghg_lulucf_17'] /25000

In [236]:
monaco = df[df['country'] == 'Monaco']

df = df = df[df.country != 'Monaco']

df

,country,core_total,mitigation_total,adaptation_total,cross_cutting_total,other_total,core_multi,mitigation_multi,adaptation_multi,cross_cutting_multi,...,ghg_17,GDP_pc_18,population,region,total_support,GDP_pc_18_stand,per_capita,gdp_18,ghg_18,stand_ghg
0,Australia,455613922.00,0.00,15611850.00,247509858.00,0.00,455613922.00,0.00,0.00,145694970.00,...,554126.56,57354.96,24982688,Oceania,718735630.00,5.74,21.40,57354.96,57354.96,21.39
1,Austria,0.00,171622069.89,32572011.17,78174346.48,0.00,0.00,1832559.10,0.00,64137273.66,...,82261.47,51453.15,8840521,Western Europe,282368427.54,5.15,8.75,51453.15,51453.15,3.09
2,Belgium,359973745.47,6969203.64,71549171.36,16714481.05,25760.96,359973745.47,1345613.50,18140259.74,180975.31,...,114539.90,47554.75,11427054,Western Europe,455232362.48,4.76,10.00,47554.75,47554.75,4.57
3,Canada,140038232.35,170608276.52,63238766.01,211061790.43,0.00,140038232.35,0.00,1549811.50,45499758.85,...,715749.23,46454.74,37065178,North America,584947065.31,4.65,18.67,46454.74,46454.74,27.68
4,Croatia,33204.07,0.00,0.00,0.00,0.00,33204.07,0.00,0.00,0.00,...,25019.75,15014.08,4087843,Southern Europe,33204.07,1.50,4.95,15014.09,15014.09,0.81
5,Cyprus,468202.37,0.00,0.00,0.00,0.00,468202.37,0.00,0.00,0.00,...,8963.24,29089.47,1189262,Southern Europe,468202.37,2.91,7.09,29089.47,29089.47,0.34
6,Czechia,7231371.00,1657304.00,4792746.00,2046140.00,0.00,7231371.00,0.00,0.00,1189931.00,...,128675.05,23419.74,10629928,Eastern Europe,15727561.00,2.34,11.90,23419.74,23419.74,5.06
7,Denmark,258702416.71,92916901.74,66160009.12,74371086.71,0.00,258702416.71,16539806.13,29543611.92,15330642.38,...,47611.52,61598.54,5793636,Northern Europe,492150414.28,6.16,8.73,61591.93,61591.93,2.02
8,Estonia,444181.22,236168.63,36929.06,862682.17,0.00,444181.22,104278.68,0.00,13672.74,...,20879.88,23159.39,1321977,Northern Europe,1579961.08,2.32,14.44,23052.30,23052.30,0.76
9,Finland,633419141.96,14598371.65,5311155.08,35071380.37,0.00,633419141.96,4027664.35,2478460.99,18085069.93,...,55334.13,50013.29,5515525,Northern Europe,688400049.06,5.00,6.34,49964.50,49964.50,1.40


In [255]:
ghg = pd.read_csv('ghg-emissions.csv')


spare = ghg[(ghg['Country/Region'].isin(['Russian Federation', 'Slovak Republic', 'Czech Republic']))]

ghg = ghg[(ghg['Country/Region'].isin(list(df.country)))]

ghg = pd.concat([ghg, spare])



ghg = ghg.sort_values('Country/Region')


ghg_18 = list(ghg['2018'])

df['ghg_18'] = ghg_18

#standard lulcf for size of scatter 
df['stand_ghg'] = df['ghg_18'] /25

In [264]:
#attempt same in bokeh

from bokeh.plotting import show, figure
from bokeh.models import ColumnDataSource, CategoricalColorMapper, LinearColorMapper,ColorBar, BasicTicker, PrintfTickFormatter, HoverTool, BasicTickFormatter, Title
from bokeh.palettes import Viridis256, d3
from bokeh.transform import transform
from bokeh.models import NumeralTickFormatter, WheelZoomTool, BoxZoomTool
from bokeh.layouts import row,column


output_file('bubble.html')


#create figure
p = figure(x_axis_label='GDP per capita in 2018', y_axis_label='$ billion climate finance, 2018', 
            sizing_mode='stretch_both', toolbar_location='below',x_range = (0, 125000),
              y_range = (0, 15000000000), min_border=0, outline_line_color = None)

p.add_layout(Title(text="Size of bubble reflects the country's total emissions* (CO2e) in 2018", text_font_style="italic",text_font_size = '7pt'), 'above')
p.add_layout(Title(text="Climate finance from rich nations"), 'above')
p.yaxis.formatter = NumeralTickFormatter(format="$0,0 a")
p.xaxis.formatter = NumeralTickFormatter(format="$0,0 a")
p.xaxis[0].ticker.desired_num_ticks = 3

# add extra titles with add_layout(...)
#p.add_layout(Title(text="Total GHG Emissions (kt)", align="right"), "above")

#set source to dataframe
source = ColumnDataSource(df)


n = len(df.region.unique())

palette = viridis(n)
color_mapper = CategoricalColorMapper(palette = palette, factors = df['region'].unique())

#scatter data
p.scatter(x = 'gdp_18', y = 'total_support', size = 'stand_ghg', color = {'field': 'region', 'transform': color_mapper}, legend_field = 'region',source = source)

#add tools
p.add_tools(HoverTool(tooltips = [('Country', '@country'), ('Total Support', '@total_support{($ 0.0 a)}'),
                                 ('GHG Emissions (millions of tonnes)', '@ghg_18{0,0}')]))


#add data sources below charts
label_opts = dict(x = 0, y = 0, x_units = 'screen', y_units ='screen')
msg = 'Data Sources: UNFCCC, the Worldbank, Climate Watch'
caption1 = Label(text=msg, **label_opts, text_font_size = '7pt', text_font_style = 'italic')
p.add_layout(caption1, 'below')

#add asterisk
msg = '*Including land use change and forestry'
caption1 = Label(text=msg, **label_opts, text_font_size = '7pt', text_font_style = 'italic')
p.add_layout(caption1, 'below')



p.xaxis.major_label_orientation = 120
p.toolbar.active_drag = None



#NOTE: There are two versions of adding the colourbar here: one with layout, one by simply adding it to the plot. 
#The layout didn't scale properly, so I opted for adding it + an extra title instead
show(p)
save (p)


'/Users/kajsarosenblad/Documents/private/Skrivet/BBC/100B/Data/bubble.html'

In [ ]:
"""
#color mapper + bar
color_mapper = LinearColorMapper(palette = Viridis256, low = df['ghg_lulucf_17'].min(), high = df['ghg_lulucf_17'].max())
color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0), label_standoff=12, ticker = BasicTicker())

color_bar.formatter.use_scientific = False
color_bar.formatter = NumeralTickFormatter(format="0,0")

"""
"""
#add bar
color_bar_plot = figure(title="Total GHG emissions (kt)", title_location="right", 
                        plot_height=750, plot_width=150, 
                        toolbar_location=None, min_border=0, 
                        outline_line_color=None)

color_bar_plot.add_layout(color_bar, 'right')
color_bar_plot.title.align="center"
color_bar_plot.title.text_font_size = '12pt'

#p.add_layout(color_bar, 'right')
"""
#layout = row(p, color_bar_plot)


In [105]:
df.head()

,country,core_total,mitigation_total,adaptation_total,cross_cutting_total,other_total,core_multi,mitigation_multi,adaptation_multi,cross_cutting_multi,...,ghg_lulucf_17,ghg_17,GDP_pc_18,population,region,total_support,GDP_pc_18_stand,per_capita,stand_ghg,gdp_17
0,Australia,455613922.00,0.00,15611850.00,247509858.00,0.00,455613922.00,0.00,0.00,145694970.00,...,534695.45,554126.56,57354.96,24982688,Oceania,718735630.00,5.74,21.40,29.71,54027.97
1,Austria,0.00,171622069.89,32572011.17,78174346.48,0.00,0.00,1832559.10,0.00,64137273.66,...,77355.42,82261.47,51453.15,8840521,Western Europe,282368427.54,5.15,8.75,4.30,47309.37
2,Belgium,359973745.47,6969203.64,71549171.36,16714481.05,25760.96,359973745.47,1345613.50,18140259.74,180975.31,...,114280.49,114539.90,47554.75,11427054,Western Europe,455232362.48,4.76,10.00,6.35,44089.31
3,Canada,140038232.35,170608276.52,63238766.01,211061790.43,0.00,140038232.35,0.00,1549811.50,45499758.85,...,692026.54,715749.23,46454.74,37065178,North America,584947065.31,4.65,18.67,38.45,45129.36
4,Croatia,33204.07,0.00,0.00,0.00,0.00,33204.07,0.00,0.00,0.00,...,20253.17,25019.75,15014.08,4087843,Southern Europe,33204.07,1.50,4.95,1.13,13451.62


In [106]:
#SECOND SCATTER OF POLLUTERS
output_file('bubble2.html')


#create figure
p = figure(title="Annex 1 countries, GHG emissions", x_axis_label='GDP per Capita', y_axis_label='Total GHG Emissions (1000 kt)', 
            sizing_mode = 'stretch_both', toolbar_location="below",x_range = (0, 90000),
              y_range = (0, 300))

p.yaxis.formatter = NumeralTickFormatter(format="0.000")
p.xaxis.formatter = NumeralTickFormatter(format="0.000")
p.xaxis[0].ticker.desired_num_ticks = 3

#add empty legend
#p.add_layout(Legend(), 'right')

#set source to dataframe
s = ColumnDataSource(df)

#color mapper 

n = len(df.region.unique())

palette = viridis(n)
color_mapper = CategoricalColorMapper(palette = palette, factors = df['region'].unique())

#scatter data
p.scatter(x = 'GDP_pc_18', y = 'stand_ghg', size = 'stand_ghg',
          color={'field': 'region', 'transform': color_mapper}, legend_field = 'region', source = s)

#add tools
p.add_tools(HoverTool(tooltips = [('Country', '@country'), ('GHG Emissions (kt)', '@stand_ghg{0.0} k'), 
                                 ('GDP per Capita', '@GDP_pc_18{$ 0.00 a}')]))
p.toolbar.active_scroll = WheelZoomTool()

#p.legend.orientation = "horizontal"
p.legend.location = "top_right"

#add data sources below charts
label_opts = dict(x = 0, y = 0, x_units = 'screen', y_units ='screen')
msg = 'Data Sources: UNFCC, the Worldbank'
caption1 = Label(text=msg, **label_opts, text_font_size = '8pt', text_font_style = 'italic')
p.add_layout(caption1, 'below')
msg1 = 'Note: Total GHG in Kiloton(kt) have been rounded to closest 10.000, to aid the visualisation.'
caption2 = Label(text=msg1, **label_opts, text_font_size = '8pt', text_font_style = 'italic')
#p.add_layout(caption2, 'below')
p.xaxis.major_label_orientation = 120
p.toolbar.active_drag = None


show(p)
save(p)


'/Users/kajsarosenblad/Documents/private/Skrivet/BBC/100B/Data/bubble2.html'

In [107]:
df.head(10)

,country,core_total,mitigation_total,adaptation_total,cross_cutting_total,other_total,core_multi,mitigation_multi,adaptation_multi,cross_cutting_multi,...,ghg_lulucf_17,ghg_17,GDP_pc_18,population,region,total_support,GDP_pc_18_stand,per_capita,stand_ghg,gdp_17
0,Australia,455613922.00,0.00,15611850.00,247509858.00,0.00,455613922.00,0.00,0.00,145694970.00,...,534695.45,554126.56,57354.96,24982688,Oceania,718735630.00,5.74,21.40,29.71,54027.97
1,Austria,0.00,171622069.89,32572011.17,78174346.48,0.00,0.00,1832559.10,0.00,64137273.66,...,77355.42,82261.47,51453.15,8840521,Western Europe,282368427.54,5.15,8.75,4.30,47309.37
2,Belgium,359973745.47,6969203.64,71549171.36,16714481.05,25760.96,359973745.47,1345613.50,18140259.74,180975.31,...,114280.49,114539.90,47554.75,11427054,Western Europe,455232362.48,4.76,10.00,6.35,44089.31
3,Canada,140038232.35,170608276.52,63238766.01,211061790.43,0.00,140038232.35,0.00,1549811.50,45499758.85,...,692026.54,715749.23,46454.74,37065178,North America,584947065.31,4.65,18.67,38.45,45129.36
4,Croatia,33204.07,0.00,0.00,0.00,0.00,33204.07,0.00,0.00,0.00,...,20253.17,25019.75,15014.08,4087843,Southern Europe,33204.07,1.50,4.95,1.13,13451.62
5,Cyprus,468202.37,0.00,0.00,0.00,0.00,468202.37,0.00,0.00,0.00,...,8429.23,8963.24,29089.47,1189262,Southern Europe,468202.37,2.91,7.09,0.47,26444.07
6,Czechia,7231371.00,1657304.00,4792746.00,2046140.00,0.00,7231371.00,0.00,0.00,1189931.00,...,126540.11,128675.05,23419.74,10629928,Eastern Europe,15727561.00,2.34,11.90,7.03,20636.20
7,Denmark,258702416.71,92916901.74,66160009.12,74371086.71,0.00,258702416.71,16539806.13,29543611.92,15330642.38,...,50582.08,47611.52,61598.54,5793636,Northern Europe,492150414.28,6.16,8.73,2.81,57610.10
8,Estonia,444181.22,236168.63,36929.06,862682.17,0.00,444181.22,104278.68,0.00,13672.74,...,19087.14,20879.88,23159.39,1321977,Northern Europe,1579961.08,2.32,14.44,1.06,20387.28
9,Finland,633419141.96,14598371.65,5311155.08,35071380.37,0.00,633419141.96,4027664.35,2478460.99,18085069.93,...,34956.41,55334.13,50013.29,5515525,Northern Europe,688400049.06,5.00,6.34,1.94,46297.50


In [108]:
#make new, smaller df

multi = df[['country',"core_multi", "mitigation_multi", 'adaptation_multi', 'cross_cutting_multi', 'other_multi']]


mitigation = multi.mitigation_multi.sum()
adaptation = multi.adaptation_multi.sum()
cross = multi.cross_cutting_multi.sum()
other = multi.other_multi.sum()

#climate specific
climate = mitigation + adaptation + cross + other

#all kinds of multilateral support
whole = multi.core_multi.sum() + climate

#core funding
core =  multi.core_multi.sum()


#set up data frame with climate specific vs core support
dff1=pd.DataFrame()

dff1['Type'] = ['Climate specific', 'Core/general funds', 'Adaptation', 'Mitigation', 'Cross cutting', 'Other']
dff1['Percentage'] = [(climate/whole), (core/whole), (adaptation/whole), (mitigation/whole), (cross/whole), (other/whole)]
dff1['Percentage'] =dff1['Percentage']*100

#set up dataframe with division of climate specific support
dff2 = pd.DataFrame()

dff2['Type'] = ['Mitigation', 'Adaptation', 'Cross cutting', 'Other']

dff2['Percentage'] = [(mitigation/climate), (adaptation/climate), (cross/climate), (other/climate)]
dff2['Percentage'] =dff2['Percentage']*100


In [109]:
dff1[:2]

,Type,Percentage
0,Climate specific,25.14
1,Core/general funds,74.86


In [110]:
dff2[:3]

,Type,Percentage
0,Mitigation,7.32
1,Adaptation,12.04
2,Cross cutting,80.63


In [111]:
dff_oecd = pd.DataFrame()

types = ['Mitigation', 'Adaptation', 'Cross cutting']
perc = [72,19,9]

dff_oecd['Type'] = types
dff_oecd['Percentage'] = perc

dff_oecd

,Type,Percentage
0,Mitigation,72
1,Adaptation,19
2,Cross cutting,9


In [265]:
#Circle graph

output_file('corevspecific.html')

#FIRST CIRCLE

x = pd.Series(dff1[:2].Percentage.values,index=dff1[:2].Type).to_dict()

data = pd.Series(x).reset_index(name='value').rename(columns={'index':'country'})
data['angle'] = data['value']/data['value'].sum() * 2*pi
data['color'] = [colors[0], colors[2]]

p = figure(sizing_mode ='stretch_both', title="How much of the finance was climate specific?", toolbar_location=None,
           tools="hover", tooltips="@country: @value{(0)}%")

#add empty legend
#p.add_layout(Legend(), 'right')

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='country', source=data)

#add data sources below charts
label_opts = dict(x = 0, y = 0, x_units = 'screen', y_units ='screen')
msg = 'Data Source: UNFCCC'
caption1 = Label(text=msg, **label_opts, text_font_size = '8pt', text_font_style = 'italic')
p.add_layout(caption1, 'below')

p.axis.axis_label=None
p.axis.visible=False
p.grid.grid_line_color = None


show(p)
save(p)


'/Users/kajsarosenblad/Documents/private/Skrivet/BBC/100B/Data/corevspecific.html'

In [113]:
#SECOND CIRCLE

output_file('adaptationmitigation.html')

x1 = pd.Series(dff_oecd.Percentage.values,index=dff_oecd.Type).to_dict()

data = pd.Series(x1).reset_index(name='value').rename(columns={'index':'country'})
data['angle'] = data['value']/data['value'].sum() * 2*pi
data['color'] = [colors[1], colors[3],colors[5]]

p1 = figure(sizing_mode='stretch_both', title="Division of climate specific finance", toolbar_location=None,
           tools="hover", tooltips="@country: @value{(0)}%")
#add empty legend
#p1.add_layout(Legend(), 'right')

p1.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='country', source=data)

p1.axis.axis_label=None
p1.axis.visible=False
p1.grid.grid_line_color = None

#add data sources below charts
label_opts = dict(x = 0, y = 0, x_units = 'screen', y_units ='screen')
msg = 'Data Source: OECD'
caption1 = Label(text=msg, **label_opts, text_font_size = '8pt', text_font_style = 'italic')
p1.add_layout(caption1, 'below')

show(p1)
save(p1)



'/Users/kajsarosenblad/Documents/private/Skrivet/BBC/100B/Data/adaptationmitigation.html'

# SCRAP

In [114]:
#trial, rezizing with python
import ctypes
screen_width = ctypes.windll.user32.GetSystemMetrics(0)
max_width = floor(96 * screen_width / 100)  # buffer to adjust
p1_width = 0.8*max_width 
p2_width = 0.2*max_width 

output_file('trial.html')

x1 = pd.Series(dff_oecd.Percentage.values,index=dff_oecd.Type).to_dict()

data = pd.Series(x1).reset_index(name='value').rename(columns={'index':'country'})
data['angle'] = data['value']/data['value'].sum() * 2*pi
data['color'] = [colors[1], colors[4],colors[6]]

p1 = figure(sizing_mode='scale_both', title="Division of Climate Specific Support", toolbar_location=None,
           tools="hover", tooltips="@country: @value{(0.00)} %", x_range=(-0.5, 1.0))
#add empty legend
p1.add_layout(Legend(), 'right')

p1.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='country', source=data)

p1.axis.axis_label=None
p1.axis.visible=False
p1.grid.grid_line_color = None

#add data sources below charts
label_opts = dict(x = 0, y = 0, x_units = 'screen', y_units ='screen')
msg = 'Data Source: OECD'
caption1 = Label(text=msg, **label_opts, text_font_size = '8pt', text_font_style = 'italic')
p1.add_layout(caption1, 'below')

show(p1)
save(p1)

AttributeError: module 'ctypes' has no attribute 'windll'

In [ ]:
#multiple lines
x = list(table1['year'])
grant = list(table1['grant'])
loan = list(table1['loan'])
other = list(table1['other'])

# create a new plot with a title and axis labels
p2 = figure(title="Instruments", x_axis_label='Year', y_axis_label='$ Billion', 
            plot_height=750, plot_width=1000, toolbar_location="below")

p2.line(x, grant, legend_label="Grant", line_width=2, color ='#31a354')
p2.line(x, loan, legend_label="Loan", line_width=2, color ='#006837')
p2.line(x, other, legend_label="Other", line_width=2, color ='#c2e699')

p2.legend.orientation = "horizontal"
p2.legend.location = "top_center"

show(p2)

In [ ]:
from bokeh.models import HoverTool

test = figure(title="Instruments", x_axis_label='Year', y_axis_label='$ Billion', 
            plot_height=750, plot_width=1000, toolbar_location = 'below')

test.multi_line(xs =[table1['year'],table1['year'],table1['year']], 
                ys= [table1['grant'], table1['loan'], table1['other']], 
                color = ['#31a354', '#006837','#c2e699'], line_width = 2, legend_label = 'Instrument')

hover = HoverTool(tooltips = [('Year', '@xs'),
                         ('Instrument', '@ys')])
test.add_tools(hover)


#multi_line(xs = [time, time], ys = [up, down], color=['#2828B0', '#BC0096'], line_width=1, legend='graph_1')
#hover = HoverTool(tooltips = [('Time', '@time'),
                         #('data', '@up')])
    
show(test)

In [ ]:
from bokeh.models import Legend

source = ColumnDataSource(data=dict(
      xs=[ x, x, x ],
      ys=[grant, loan, other],
    color= ['#006837', '#c2e699', '#78c679'],
    labels = ['Grant', 'Loan', 'Others']
  ))

TOOLTIPS = [
    ("Year", "@xs"),
    ("USD", "@ys"),
]

f = figure(title="Instruments", x_axis_label='Year', y_axis_label='$ Billion', 
            plot_height=750, plot_width=1000, toolbar_location="below")

f.multi_line(xs='xs', ys='ys', legend = 'labels', source=source, line_color = 'color', line_width = 3)


f.legend.orientation = "horizontal"
f.legend.location = "top_center"

hover_tool = HoverTool( tooltips=[( 'Name ', '@ys')])
f.add_tools(hover_tool)


show(f)

In [ ]:
df_trial = df.copy()

In [ ]:
df_trial.head()

In [ ]:
print('Total core: ',df_trial['core_total'].sum())
print('Total mitigation: ', df_trial['mitigation_total'].sum())
print('Total adaptation: ', df_trial['adaptation_total'].sum())